In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Jul 16 18:37:29 2023

@author: shjo9
"""
import matplotlib
matplotlib.use('agg')
import sys
sys.path.append('C:/Users/shjo/Bridge/JNUpack/SO/libs/')
import os
import numpy as np
import xarray as xr
from eofs.xarray import Eof
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
from myPlot import  figmaster,myClrbr
import cartopy.feature as cf
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import FormatStrFormatter
# from eofs.standard import Eof 
NN=10
pth='J:/MDLS_OBS_OHC/Area_data/'
#ncname='GECCO_OHC_SO_c14_2000m_1980_2018.nc'
wpth='J:/MDLS_OBS_OHC/Area_data/tmp2/'
Dir_pth='EOF_IAP_ohc700m_1Y'

try:
    os.mkdir(wpth+Dir_pth)
    os.mkdir(wpth+Dir_pth+'/ppt')
except:
    pass

# CMAP=plt.get_cmap('RdYlBu_r',15)
CNN=21
fac=100
Mylim=[-0.04*fac,0.04*fac]

My_levels=np.linspace(Mylim[0], Mylim[-1], CNN+1,endpoint=True)
# My_levels=np.arange(Mylim[0], Mylim[-1], 0.2)


Mycolorlist=['#1b2c62','#1f4181','#2455a1','#3877ba','#529bd2','#71b8e4','#91d2f2','#b2e0fa','#cbebf8','#e3f4fb',\
 '#f2f9e3','#fcf0b4','#fddb81','#fdc152','#fca12f','#f8822b','#ef5e29','#e03b28','#cc1e27','#ae191f']      
Mycmap = LinearSegmentedColormap.from_list('',Mycolorlist,N=256)
CMAP = ListedColormap(Mycmap(
    np.linspace(0, 1, len(My_levels)-1,endpoint=True)) )



OHC = xr.open_mfdataset(pth+'myIAP_198001_201812_tshuv.nc').OHC700.loc[dict(lat=slice(-80,-10))]
    
OHC_2Y=OHC.rolling(time=12,center=True).mean()[6:-5]
lonR,latR=np.meshgrid(OHC_2Y.lon.values,OHC_2Y.lat.values)

#lon_m,lat_m=np.meshgrid(OHC.lon,OHC.lat)
# coslat=np.cos(np.deg2rad(lat_m)) 
# wgts=np.sqrt(coslat)[:,np.newaxis] 
# solver=Eof(OHC.OHC,weights=wgts)

solver=Eof(OHC_2Y)
eofs = -solver.eofs(neofs=NN, eofscaling=0)
pcs = -solver.pcs(npcs=NN,pcscaling=0)

var_=solver.varianceFraction(NN)
var=var_/np.sum(var_)*100

mySetting={
    'figsize': '',
    'mylabel': '',
    'Label_size':18,
    'title_loc':'right',
    'fontParams':'Arial',
    'wpth':wpth}

### Figure ==============================================================
print('!!!   figure   !!!')
F=figmaster(mySetting)

def plot_pcs(time,time2,pc,t_name,w_path,save_name,fig_bool=True):
    Label_size = 18
    fig, axs = plt.subplots(1,1,figsize=(10,3.7),constrained_layout = True,
                        dpi=200)
    f1 = axs.plot(time,pc, label='KINETIC_ENRG',color='k',linewidth=2,zorder=0)
    axs.set_title(t_name,loc='right',fontdict={'fontsize':20,'fontweight':'regular','fontstyle':'italic'})
    axs.tick_params(axis='both', labelsize=Label_size)
    axs.grid(axis='x',linestyle='-.')
    xtick_location = time[5::12*4]
    xtick_labels = time2[5::12*4]
    axs.set_xticks(ticks=xtick_location)
    axs.set_xticklabels(xtick_labels, rotation=0, fontsize=Label_size, alpha=1)
    axs.tick_params(axis='x', direction='in', length=6, pad=8, labelsize=Label_size, labelcolor='k', top=True,width=1.)
    axs.tick_params(axis='y', direction='in', length=6, pad=8, labelsize=Label_size-3, width=1., color='k')
    plt.tight_layout()
    if fig_bool:
        plt.savefig(w_path+Dir_pth+'/ppt/'+save_name,
                facecolor='none',edgecolor='none',bbox_inches='tight',transparent=True)
        plt.savefig(w_path+Dir_pth+'/'+save_name,bbox_inches='tight')
    plt.show()
plt.rcParams["font.family"] = 'Arial'
### Plot eof ==================================================================
for i,j,n,m in zip(eofs[0:10].values*fac,np.arange(1,11),var.values,var_.values*100):
    save_name=Dir_pth+'_'+f'{j:02d}'+'mode'
    t_name='~700m '+f'{j:02d}'+' mode '+f'{n:.1f}'+'% ('+f'{m:.1f}'+'%)'
    # i[i>Mylim[-1]]=Mylim[-1]
    # i[i<Mylim[0]]=Mylim[0]
    # Plot_SO_Merc3(eofs.lon,eofs.lat,i,t_name,CMAP,Mylim,My_levels,wpth,save_name,fig_bool=True)
    # Plot_SO_Merc3(eofs.lon,eofs.lat,-i,t_name,CMAP,Mylim,My_levels,wpth,save_name+'_re',fig_bool=True)
    
    F.myCrtpy_sph(latR,lonR,i,CMAP,My_levels,save_name)
    F.myCrtpy_sph(latR,lonR,-i,CMAP,My_levels,save_name+'_re')
'''
### Plot pcs ==================================================================
TIME= [str(i)[0:7] for i in pcs.time.values]
TIME2=[str(i)[2:4] for i in pcs.time.values]
w_path=wpth
for i,j,n,m in zip(pcs.values.transpose(),np.arange(1,11),var.values,var_.values*100):
    save_name='PC_'+f'{j:02d}'+'mode'
    t_name='~700m '+f'{j:02d}'+' mode '+f'{n:.1f}'+'% ('+f'{m:.1f}'+'%)'
    plot_pcs(TIME,TIME2,i,t_name,w_path,save_name,fig_bool=True)
    plot_pcs(TIME,TIME2,-i,t_name,w_path,save_name+'_re',fig_bool=True)
# IPA_2000={'eofs':eofs.values,'pcs':pcs.values,'val':var.values,'val_':var_.values}

# np.save('D:/HEAT/DATA/EOFs/EOF_IAP_2000m.npy',IPA_2000)
'''



c:\Users\shjo\miniconda3\envs\nemo_env\lib\site-packages\xarray\backends\plugins.py:80: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


!!!   figure   !!!


c:\Users\shjo\miniconda3\envs\nemo_env\lib\site-packages\cartopy\mpl\gridliner.py:451: UserWarning: The .xlabels_top attribute is deprecated. Please use .top_labels to toggle visibility instead.
  warnings.warn('The .xlabels_top attribute is deprecated. Please '
c:\Users\shjo\miniconda3\envs\nemo_env\lib\site-packages\cartopy\mpl\gridliner.py:487: UserWarning: The .ylabels_right attribute is deprecated. Please use .right_labels to toggle visibility instead.
  warnings.warn('The .ylabels_right attribute is deprecated. Please '
c:\Users\shjo\miniconda3\envs\nemo_env\lib\site-packages\cartopy\mpl\geoaxes.py:1785: UserWarning: shading value 'gauround' not in list of valid values ['gouraud', 'nearest', 'flat', 'auto']. Setting shading='auto'.
  result = super().pcolormesh(*args, **kwargs)
C:\Users/shjo/Bridge/JNUpack/SO/libs\myPlot.py:162: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
c:\Users\shjo\miniconda3\envs\nemo_en

"\n### Plot pcs ==================================================================\nTIME= [str(i)[0:7] for i in pcs.time.values]\nTIME2=[str(i)[2:4] for i in pcs.time.values]\nw_path=wpth\nfor i,j,n,m in zip(pcs.values.transpose(),np.arange(1,11),var.values,var_.values*100):\n    save_name='PC_'+f'{j:02d}'+'mode'\n    t_name='~700m '+f'{j:02d}'+' mode '+f'{n:.1f}'+'% ('+f'{m:.1f}'+'%)'\n    plot_pcs(TIME,TIME2,i,t_name,w_path,save_name,fig_bool=True)\n    plot_pcs(TIME,TIME2,-i,t_name,w_path,save_name+'_re',fig_bool=True)\n# IPA_2000={'eofs':eofs.values,'pcs':pcs.values,'val':var.values,'val_':var_.values}\n\n# np.save('D:/HEAT/DATA/EOFs/EOF_IAP_2000m.npy',IPA_2000)\n"